In [1]:
import tensorflow as tf

In [2]:
vgg = tf.keras.applications.VGG19()
vgg.summary()

W1119 23:01:25.373756  3696 deprecation.py:506] From C:\Users\Gencho\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
#input_tensor = tf.keras.Input(shape = (224, 224, 3))
vgg = tf.keras.applications.VGG19(weights ='imagenet')

In [5]:
vgg.trainable = False
vgg.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
labels = 2 # dog, cat
grid_size = 13 # grid is the size squared
boxes_per_grid_cell = 2

In [7]:
layers = vgg.layers[:-1]
object_detection_predictions_layer = tf.keras.layers.Dense(units = grid_size * grid_size * (5 * boxes_per_grid_cell + labels))
#layers.append(tf.keras.layers.Flatten())
layers.append(object_detection_predictions_layer)

model = tf.keras.Sequential(layers, "object detector")
model.summary()

Model: "object detector"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)   

In [8]:
'''
notes:
    normalize bounding box width and height according to image width and height so that they fall between 0 and 1

'''

def get_expected_output_from_annotations(annotation_path):
    #TODO: Implement
    
    pass

def get_predictions_from_output(output):
    #TODO: Implement
    
    pass

In [9]:
Lcoord = 5
Lnoobj = 0.5

In [18]:
def loss(y_true, y_pred):
    '''
    output tensor:
    
    for each cell:
        for each bounding box:
            x, y, w, h, C(any)
        for each class:
            C(class)
    
    flattened
    '''
    from math import sqrt
    
    loss = 0
    
    step_size = 5 * boxes_per_grid_cell + labels # 12
    
    def get_true_pred(index):
        return y_true[index], y_pred[index]
    
    for cell in range(grid_size ** 2):
        index = cell * step_size # 0
        
        
        for box in range(boxes_per_grid_cell):
            inner_step_size = box * 5
            
            
            xtrue, xpred = get_true_pred(inner_step_size + index)
            ytrue, ypred = get_true_pred(inner_step_size + index + 1)
            wtrue, wpred = get_true_pred(inner_step_size + index + 2)
            htrue, hpred = get_true_pred(inner_step_size + index + 3)
            Canytrue, Canypred = get_true_pred(inner_step_size + index + 4)
                                  
            
            loss += (Lnoobj if Canytrue == 0 else 1) * ((Canytrue - Canypred) ** 2)
            
                    
            if Canytrue:
                loss += Lcoord * ((xtrue - xpred) ** 2 + (ytrue - ypred) ** 2)
                loss += Lcoord * ((sqrt(wtrue) - sqrt(wpred)) ** 2 + (sqrt(htrue) - sqrt(hpred)) ** 2)
        
        for label in range(labels):
            Clabeltrue, Clabelpred = get_true_pred(index + boxes_per_grid_cell * 5 + label)
            
            loss += (Lnoobj if Clabeltrue == 0 else 1) * ((Clabeltrue - Clabelpred) ** 2)
                
    
    pass

In [19]:
model.compile(optimizer = "adam", loss = loss)

TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.